## Student: Huy Hoang Le

In [13]:
from pathlib import Path

import numpy as np
import cv2 as cv

class Paths:
    DIR_RESULT = Path("results")

    @staticmethod
    def path_ex53(i: int):
        path = Paths.DIR_RESULT / "5_3" / f"{i:04}.jpg"
        path.parent.mkdir(exist_ok=True, parents=True)

        return path
    
    @staticmethod
    def path_ex54(i: int):
        path1 = Paths.DIR_RESULT / "5_4" / f"{i:04}.jpg"
        path2 = Paths.DIR_RESULT / "5_4" / f"{i:04}_mask.jpg"
        path1.parent.mkdir(exist_ok=True, parents=True)

        return path1, path2

## Exercise 5.1.

### Answer

(1)

(2) Steps to rectify stereo images

Requirement: 2 cameras are calibrated. I assume 2 cameras have the same calibration matrix $K$ for simplicity.

Denote:
- $K$: calibration matrix
- $P_1$, $P_2: camera matrix of 2 cameras
- $F$: fundamental matrix
- $R$, $t$: rotation matrix and translation vector indicating displacement from camera 2 to the world origin (i.e. camera 1)


Assume:
- world origin is located at the 1st camera

Steps:
1. Find Fundamental matrix $F$ with 8-point algorithm, then determine $R$ and $t$
2. Rotate right image with $R$
3. Rotate both images with $R$ and $R_{rect}$
   ($R_{rect}$ is defined such that after this step, the epipolar lines of both images are parallel with the baseline and coincided)
4. Scale left image with $KR_{rect}$, scale right image with $KRR_{rect}$
   (This scaling step helps minimizing horizontal distortion)

## Exercise 5.2.

### Answer

Lucas-Kanade algorithm is a patch-based solution to find the displacement. It works based on several assumptions:
- The flow is locally smooth
- The displacement is similar with all pixels withing a patch.



## Exercise 5.3.

In [6]:


path = "OF_Single.mp4"

cap = cv.VideoCapture(path)

 
# params for ShiTomasi corner detection
feature_params = dict(
    maxCorners = 100,
    qualityLevel = 0.3,
    minDistance = 7,
    blockSize = 7
)

# Parameters for lucas kanade optical flow
lk_params = dict(
    winSize  = (15, 15),
    maxLevel = 2,
    criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03)
)
 
# Create some random colors
color = np.random.randint(0, 255, (100, 3))
 
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

width, height,_ = old_frame.shape
vidwriter = cv.VideoWriter("output.mp4", cv.VideoWriter_fourcc(*"mp4v"), 25, (width, height))
 
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
i = 1
while True:
    ret, frame = cap.read()
    if not ret:
        print('here: No frames grabbed!')
        break
 
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
 
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
 
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
 
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv.add(frame, mask)
 
    # cv.imshow('frame', img)
    # k = cv.waitKey(30) & 0xff
    # if k == 27:
    #     break
    cv.imwrite(Paths.path_ex53(i), img)
 
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

    i += 1


No frames grabbed!


### Answer

(2)

The reason why Lucas-Kande only chooses corners to track is to avoid aperture problem. Basically, if a point is in a line and the movement is across the line, it is very hard to determine the displacement. Therefore, by choosing corners, it allows the displacement can perform in any direction.

[?] Moreover, some selected pixels are lost or changed in the tracking process. Please explain the reason.
The


## Exercise 5.4.

In [14]:
path = "OF_Dense.mp4"

cap = cv.VideoCapture(path)

ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
i = 1
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
 
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    # cv.imshow('frame2', bgr)
    # cv.imwrite(, bgr)
    path_img, path_msk = Paths.path_ex54(i)
    cv.imwrite(path_img, frame2)
    cv.imwrite(path_msk, bgr)

    prvs = next

    i += 1

No frames grabbed!
